# RESNET MODEL

In [ ]:
#working model - Train

In [ ]:
# Define paths
cropped_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test/test_cropped/cropped"
output_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/output/"

train_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/train/train"
test_data_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test"

validated_images_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/test/test_cropped/validated"

processed_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/processed/"
error_folder = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/result/error/"

class_labels_file = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/class_labels.pkl"
model_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/resnet_model.h5"

In [1]:
import cv2
import numpy as np
import os
import pickle
import time
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam

# Define constants
training_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/train/train"
trained_model_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/"
class_labels_path = "C:/Users/Learning/udemy/Hackathon/ultimate/ResNet/dataset/trained_models/class_labels.pkl"

# Function to resize an image
def resize_image(image, target_width=500):
    aspect_ratio = image.shape[1] / image.shape[0]
    target_height = int(target_width / aspect_ratio)
    resized_image = cv2.resize(image, (target_width, target_height))
    return resized_image

# Define ResNet model
def create_resnet_model(input_shape, num_classes):
    model = Sequential()
    # Define model layers
    model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Add a fully connected layer with softmax activation for classification
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze all layers in the base model
    for layer in base_model.layers:
        layer.trainable = False
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Dimensions of input images
input_shape = (100, 100, 3)
# Number of classes
num_classes = 18

# Build the ResNet model
model = build_resnet_model(input_shape, num_classes)

def load_images_and_labels(path_dataset, width, height):
    images = []
    labels = []

    subdirs = [os.path.join(path_dataset, f) for f in os.listdir(path_dataset)]

    for subdir in subdirs:
        name = subdir.split(os.path.sep)[-1]
        images_list = [os.path.join(subdir, f) for f in os.listdir(subdir) if not os.path.basename(f).startswith(".")]

        for image_path in images_list:
            try:
                img = cv2.imread(image_path)
                if img is None:
                    print("Error loading image:", image_path)
                    continue

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (width, height))  # Resize image to a fixed size
                images.append(img)
                labels.append(name)
            except Exception as e:
                print("Error processing image:", image_path)
                print(e)

    return images, labels

def train_resnet_model(images_train, labels_train, images_val, labels_val):
    target_width = 100
    target_height = 100
    images_train_resized = [resize_image(img, target_width) for img in images_train]
    images_val_resized = [resize_image(img, target_width) for img in images_val]

    input_shape = images_train_resized[0].shape
    num_classes = len(np.unique(labels_train))
    model = create_resnet_model(input_shape, num_classes)
    
    label_to_index = {label: i for i, label in enumerate(np.unique(labels_train))}
    encoded_labels_train = to_categorical([label_to_index[label] for label in labels_train], num_classes=num_classes)
    
    # Filter validation set labels to include only those present in the training set
    filtered_labels_val = [label for label in labels_val if label in label_to_index]
    encoded_labels_val = to_categorical([label_to_index[label] for label in filtered_labels_val], num_classes=num_classes)
    
    train_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow(np.array(images_train_resized), encoded_labels_train, batch_size=32)
    
    # Filter validation images to match filtered labels
    filtered_images_val = [image for image, label in zip(images_val_resized, labels_val) if label in label_to_index]
    val_datagen = ImageDataGenerator(rescale=1./255)
    val_generator = val_datagen.flow(np.array(filtered_images_val), encoded_labels_val, batch_size=32)
    
    # Train the model
    model.fit(train_generator, validation_data=val_generator, epochs=10)
    
    return model, label_to_index

start_time = time.time()
print("Execution started...")

# Load images and labels
images, labels = load_images_and_labels(training_path, width=100, height=100)

# Split data into training and validation sets
images_train, images_val, labels_train, labels_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Ensure that the number of images matches the number of labels
assert len(images_train) == len(labels_train)
assert len(images_val) == len(labels_val)

# Train ResNet model
model, index_to_label = train_resnet_model(images_train, labels_train, images_val, labels_val)

# Save the trained model
model.save(trained_model_path + "resnet_model.h5")

# Save the index to label mapping
with open(class_labels_path, "wb") as f:
    pickle.dump(index_to_label, f)

print("Index to label mapping saved at:", class_labels_path)
print("Execution completed.")
print("Total execution time: {:.2f} seconds".format(time.time() - start_time))





Execution started...

Epoch 1/10


4/4 [==============================] - 3s 348ms/step - loss: 1.5806 - accuracy: 0.7739 - val_loss: 0.6354 - val_accuracy: 0.8966
Epoch 2/10
4/4 [==============================] - 1s 196ms/step - loss: 0.6493 - accuracy: 0.8522 - val_loss: 0.3998 - val_accuracy: 0.8966
Epoch 3/10
4/4 [==============================] - 1s 205ms/step - loss: 0.5144 - accuracy: 0.8609 - val_loss: 0.2584 - val_accuracy: 0.8966
Epoch 4/10
4/4 [==============================] - 1s 195ms/step - loss: 0.4153 - accuracy: 0.8696 - val_loss: 0.1789 - val_accuracy: 0.8966
Epoch 5/10
4/4 [==============================] - 1s 228ms/step - loss: 0.2998 - accuracy: 0.8783 - val_loss: 0.1538 - val_accuracy: 0.8966
Epoch 6/10
4/4 [==============================] - 1s 214ms/step - loss: 0.2580 - accuracy: 0.9130 - val_loss: 0.0863 - val_accuracy: 0.9655
Epoch 7/10
4/4 [==============================] - 1s 209ms/step - loss: 0.2147 - accuracy: 0.9043 - val_loss: 0.0510 - val_accuracy: 

C:\Users\Learning\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
